## Etape 1: récupérez les données et commencez à les regarder 

1. Créez une base sqlite localement
2. Créez deux tables et uploadez les données
3. Récupérez les tables sous forme de dataframe python et faites une exploration rapide de ces données.
4. Quels sont selon vous dans les données patients les données personnelles (identificateurs & quasi-identificateurs) et les données sensibles ?
5. De combien de patients retrouvez vous le nom en faisant une jointure entre ces deux tables ?

### Soluce

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sqlite3
from datetime import date
from dateutil.relativedelta import relativedelta

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches

from anonymity.mondrian import MondrianAnonymizer, is_k_anonymous

sns.set_style("whitegrid")

1. Créez une base sqlite localement

In [ ]:
db_filepath = Path('tp_anonymisation.db')
db_filepath.unlink(missing_ok=True) # rm db if exists

In [ ]:
conn = sqlite3.connect(db_filepath)
c = conn.cursor()

2. Créez deux tables et uploadez les données

Avec sqlite3

```
.separator ;
.import personal_data.csv personal_data
.import sensitive_data.csv sensitive_data
.mode list
```

Avec python / pandas

In [ ]:
items_to_feed = [('personal_data.csv', 'personal_data'),
                 ('sensitive_data.csv', 'sensitive_data')]

for filepath, table_name in items_to_feed:
    pd.read_csv(filepath, sep=';')\
        .to_sql(table_name, conn, if_exists='replace', index=False)

Récupérez les tables sous forme de dataframe python et faites une exploration rapide de ces données.

In [ ]:
q = '''
select *
from sensitive_data
'''
df_sdata = pd.read_sql(q, conn)

Nombre de patients dans notre fichier de données:

In [ ]:
len(df_sdata)

In [ ]:
df_sdata.head()

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(10, 10))

# age distribution
ax00 = ax[0, 0]
df_sdata['age'].hist(ax=ax00)
ax00.set_xlabel('age')


# sex "ratio"
ax01 = ax[0, 1]
df_sdata['sexe'].value_counts().plot(ax=ax01, kind='bar')

# repartition selon le département
ax10 = ax[1, 0]
df_sdata['code_postal'].apply(lambda x: str(x)[:2]).value_counts().plot(ax=ax10, kind='bar')

# tension arterielle
ax11 = ax[1, 1]
df_sdata['tension_arterielle'].value_counts().plot(ax=ax11, kind='barh')

# tension arterielle
ax21 = ax[2, 0]
df_sdata['consommation_alcool'].value_counts().plot(ax=ax21, kind='barh')

# tension arterielle
ax22 = ax[2, 1]
df_sdata['activite_physique'].value_counts().plot(ax=ax22, kind='barh')

fig.tight_layout()

In [ ]:
q = '''
select *
from personal_data
'''
df_pdata = pd.read_sql(q, conn)

Nombre de personnes dans notre fichier de données marketting:

In [ ]:
len(df_pdata)

In [ ]:
df_pdata.head()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))

# age distribution
ax00 = ax[0, 0]
df_pdata['age'].hist(ax=ax00)
ax00.set_xlabel('age')

# sex "ratio"
ax01 = ax[0, 1]
df_pdata['sexe'].value_counts().plot(ax=ax01, kind='bar')

# repartition according to department
ax10 = ax[1, 0]
df_pdata['code_postal'].apply(lambda x: str(x)[:2]).value_counts().plot(ax=ax10, kind='bar')

fig.tight_layout()

4. Quels sont selon vous dans les données patients les données personnelles (identificateurs & quasi-identificateurs) et les données sensibles ?
5. De combien de patients retrouvez vous le nom en faisant une jointure entre ces deux tables ?

In [ ]:
q = '''
select sdata.*
from sensitive_data as sdata
inner join personal_data as pdata
on pdata.nom = sdata.nom and pdata.prenom = sdata.prenom and pdata.adresse = sdata.adresse and pdata.age=sdata.age and pdata.sexe=sdata.sexe;
'''
df_join = pd.read_sql(q, conn)

Nombre de patients de notre fichier de données qui correspondent (nom, prenom, adresse) à des personnes issues du fichier de données marketting:

In [ ]:
len(df_join)

In [ ]:
df_join.head()

In [ ]:
# attention aux homonymes
q = '''
select sdata.*
from sensitive_data as sdata
inner join personal_data as pdata
on pdata.nom = sdata.nom and pdata.prenom = sdata.prenom and pdata.adresse = sdata.adresse
group by pdata.nom, pdata.prenom, pdata.adresse
having count(*) > 1
'''
pd.read_sql(q, conn)

Nombre de patients de notre fichier de données qui correspondent (nom, prenom, adresse) à des personnes issues du fichier de données marketting (sans ambiguité):

In [ ]:
len(df_join)

## Etape 2: Pseudonymisez vos données puis attaquez les

1. Supprimer les identificateurs et les remplacer par une valeur unique par patient.
2. Chargez les données pseudonymisées. Combien d'individus vous arrivez à identifier de manière unique et pour combien retrouvez vous le nom et le prénom ?
    * Quelles sont selon vous les limites de la pseudonymisation ?
    * Que feriez vous pour améliorer cette approche ?
3. Mettez en oeuvre vos idées. Pour rappel, dans le cadre de votre étude, il vous faut conserver au minimum sujet des patients:
    * une idée de l'âge
    * le genre
    * une idée de la localisation géographique

### Soluce

1. Supprimer les identificateurs et les remplacer par une valeur unique par patient.

In [ ]:
del df_join['nom']
del df_join['prenom']

2. Chargez les données pseudonymisées. Combien d'individus vous arrivez à identifier de manière unique et pour combien retrouvez vous le nom et le prénom ?
    * Quelles sont selon vous les limites de la pseudonymisation ?
    * Que feriez vous pour améliorer cette approche ?

In [ ]:
q = '''
create table  IF NOT EXISTS sensitive_data_pseudo
as 
select sexe,
       age,
       adresse,
       code_postal,
       date_naissance,
       activite_physique,
       consommation_alcool,
       tension_arterielle
       from sensitive_data 
'''
c.execute(q)
conn.commit()

In [ ]:
q = '''
select *
from sensitive_data_pseudo 
group by age, sexe, adresse
having count(*) == 1
'''

df_sdata_pseudo = pd.read_sql(q, conn)

In [ ]:
df_sdata_pseudo

In [ ]:
q = '''
with all_matches as (
select pdata.nom,
       pdata.prenom,
       sdata.*
from sensitive_data_pseudo as sdata
inner join personal_data as pdata
on pdata.age = sdata.age and pdata.adresse = sdata.adresse and pdata.sexe = sdata.sexe
)
select *
from all_matches
group by age, adresse, sexe
having count(*) == 1
'''
df_pseudo_identified = pd.read_sql(q, conn)

Nombre de patients de notre fichier de données pseudonymisé qui correspondent (nom, prenom, adresse) à des personnes issues du fichier de données marketting (sans ambiguité):

In [ ]:
len(df_pseudo_identified)

In [ ]:
q = '''
with all_matches as (
select pdata.nom,
       pdata.prenom,
       sdata.*
from sensitive_data_pseudo as sdata
inner join personal_data as pdata
on pdata.age = sdata.age and pdata.adresse = sdata.adresse and pdata.sexe = sdata.sexe
),

ambiguous as (
select age, adresse, sexe
from all_matches
group by age, adresse, sexe
having count(*) > 1
)
select all_matches.*
from all_matches
join ambiguous
on all_matches.age = ambiguous.age and all_matches.sexe = ambiguous.sexe and all_matches.adresse = ambiguous.adresse
'''
df_pseudo_unidentified = pd.read_sql(q, conn)

In [ ]:
df_pdata.groupby(['adresse', 'age', 'sexe']).size().value_counts()

3. Mettez en oeuvre vos idées. Pour rappel, dans le cadre de votre étude, il vous faut conserver au minimum sujet des patients:
    * une idée de l'âge
    * le genre
    * une idée de la localisation géographique

In [ ]:
df_sensitive_pseudo = pd.read_sql('select * from sensitive_data_pseudo', conn)

In [ ]:
s_age = df_sensitive_pseudo['age']\
        .replace('100 ou plus', '100')\
        .astype(int)
df_sensitive_pseudo['age_cut'] = pd.cut(s_age, range(0, 110, 10))
del df_sensitive_pseudo['date_naissance']
del df_sensitive_pseudo['adresse']
del df_sensitive_pseudo['age']

In [ ]:
df_personal_data = pd.read_sql('select * from personal_data', conn)

In [ ]:
s_age = df_personal_data['age']\
        .astype(int)
df_personal_data['age_cut'] = pd.cut(s_age, range(0, 110, 10))

In [ ]:
df_sensitive_pseudo_identifiable = df_sensitive_pseudo[~df_sensitive_pseudo.duplicated(['age_cut', 'code_postal', 'sexe'], keep=False)]

Nombre de patients réidentifiables (identifiés de manière unique dans les données de santé) à partir de l'age (discretisé) du code postal et du sexe:

In [ ]:
len(df_sensitive_pseudo_identifiable)

In [ ]:
df_merge = pd.merge(df_sensitive_pseudo_identifiable,
                    df_personal_data,
                    on=('age_cut', 'code_postal', 'sexe'))

Nombre de patients réidentifiés à partir de l'age (discretisé) du code postal et du sexe:

In [ ]:
len(df_merge) - df_merge.duplicated(['age_cut', 'code_postal', 'sexe'], keep=False).sum()

In [ ]:
df_merge_identified = df_merge[~df_merge.duplicated(['age_cut', 'code_postal', 'sexe'], keep=False)]

In [ ]:
df_merge_identified

## Anonymisation

1. Déterminez le paramètre K associé à votre anonymisation
2. 10-anonymisez ces données (manuellement ou pensez à un algorithme simple pour le faire)
3. En vous servant des différentes limites de la K-anonymisation, tâchez de retrouver le plus d'informations possibles sur les patients du jeu de données K-anonymisé

### Soluce

1. Déterminez le paramètre K associé à votre anonymisation

In [ ]:
s = df_sensitive_pseudo.groupby(['sexe', 'code_postal', 'age_cut']).size()
s[s>0].min()

2. 4-anonymisez ces données (manuellement ou pensez à un algorithme simple pour le faire)

In [ ]:
del df_sdata['nom']
del df_sdata['prenom']
del df_sdata['adresse']

In [ ]:
df_sdata['code_postal'] = df_sdata['code_postal'].astype(int)
df_sdata['age'] = df_sdata['age'].astype(int)

In [ ]:
feature_columns = ['age', 'code_postal']
categorical = []
sensitive_columns = ['activite_physique', 'consommation_alcool', 'tension_arterielle']
no_agg_features = ['sexe']

anonymizer = MondrianAnonymizer(is_valid_func=is_k_anonymous, k=4)
df_sdata_h = df_sdata.query('sexe == 1')
df_sdata_f = df_sdata.query('sexe == 2')

df_k_anonymised = anonymizer.anonymize(df_sdata, feature_columns, categorical, sensitive_columns, no_agg_features)
df_k_anonymised_h = df_k_anonymised.query('sexe == 1')
df_k_anonymised_f = df_k_anonymised.query('sexe == 2')

In [ ]:
df_k_anonymised_h.groupby(['sexe', 'code_postal', 'age']).size().to_frame().rename(columns={0: 'size'}).min()

In [ ]:
df_k_anonymised_f.groupby(['sexe', 'code_postal', 'age']).size().to_frame().rename(columns={0: 'size'}).min()

3. En vous servant des différentes limites de la K-anonymisation, tâchez de retrouver le plus d'informations possibles sur les patients du jeu de données K-anonymisé

In [ ]:
df_dummies = pd.get_dummies(df_k_anonymised[sensitive_columns])
df_dummies.rename(columns={"activite_physique": "activite_physique_suffisante"}, inplace=True)
df_dummies['activite_physique_insuffisante'] = 1 - df_dummies.activite_physique_suffisante

sensitive_columns_dummies = df_dummies.columns
df_hm = pd.concat(
    [df_k_anonymised[['sexe', 'code_postal', 'age']],
     df_dummies],
     axis=1)

In [ ]:
df_hm = df_hm.groupby(['sexe', 'code_postal', 'age'])[sensitive_columns_dummies].mean()
fig, ax = plt.subplots(figsize=(20, 30))
sns.heatmap(df_hm[(df_hm == 1).any(axis=1)], ax=ax, annot=True)

In [ ]:
def plot_mondrian(df, df_k_anonym):
    fig, ax = plt.subplots(figsize=(20, 10))
    df_plot = df\
                .groupby(['age', 'code_postal'])\
                .size()\
                .to_frame()\
                .reset_index()\
                .rename(columns={0: 'n_patients'})

    sc = ax.scatter(x=df_plot.age, y=df_plot.code_postal, s=15, c=df_plot.n_patients, cmap='hot')

    for (age, code_postal), dfx in df_k_anonym.groupby(['age', 'code_postal']):
        if '-' in str(age):
            age_min, age_max = map(int, age.split('-'))
        else:
            age = int(age)
            age_min, age_max = age, age

        age_min -= 0.5
        age_max += 0.5

        if '-' in str(code_postal):
            code_postal_min, code_postal_max = map(lambda x: int(x), code_postal.split('-'))
            code_postal_jitter = .5
        else:
            code_postal = int(code_postal)
            code_postal_min, code_postal_max  = code_postal, code_postal
            code_postal_jitter = .01 * (code_postal - 69000)
        
        code_postal_min -= code_postal_jitter
        code_postal_max += code_postal_jitter

        rect = ax.add_patch(
        patches.Rectangle(
            (age_min, code_postal_min),
            age_max - age_min,
            code_postal_max - code_postal_min,
            facecolor = 'red',
            edgecolor='red',
            fill=True,
            alpha=0.2
         ) )

    # ax.set_yscale('log')
    fig.colorbar(sc, ax=ax)

In [ ]:
plot_mondrian(df_sdata_h, df_k_anonymised_h)

In [ ]:
plot_mondrian(df_sdata_f, df_k_anonymised_f)